# **Programa de Pós-Graduação em Computação - INF/UFRGS**
### Disciplina CMP263 - Aprendizagem de Máquina
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br>

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas  e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de algoritmos e estratégias de avaliação em Aprendizado de Máquina.*


---

##**Tópico: Introdução à avaliação de modelos com Holdout**


**Objetivos da atividade:**
-  Entender o funcionamento da técnica de holdout para avaliação de modelos.
- Comparar o desempenho de dois algoritmos, Naive Bayes e KNN, em uma tarefa de classificação.
- Aplicar métricas de avaliação como acurácia, precisão, recall, observando a matriz de confusão.
- Implementar uma estratégia básica para seleção de hiperparâmetros para o KNN, interpretando os resultados.
- Analisar o impacto de aspectos como dimensão dos dados, aleatoriedade na divisão de dados e repetições na avaliação de modelos com Holdout.

###Carregando as bibliotecas e dados


In [ ]:
import pandas as pd             # biblioteca para análise de dados
import matplotlib.pyplot as plt # biblioteca para visualização de informações
import seaborn as sns           # biblioteca para visualização de informações
import numpy as np              # biblioteca para operações com arrays multidimensionais
from sklearn.datasets import load_breast_cancer ## conjunto de dados a ser analisado
sns.set()

In [ ]:
## Carregando os dados - Câncer de Mama
## https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

data = load_breast_cancer() ## carrega os dados de breast cancer
X = data.data  # matriz contendo os atributos
y = data.target  # vetor contendo a classe (0 para maligno e 1 para benigno) de cada instância
feature_names = data.feature_names  # nome de cada atributo
target_names = data.target_names  # nome de cada classe

In [ ]:
print(f"Dimensões de X: {X.shape}\n")
print(f"Dimensões de y: {y.shape}\n")
print(f"Nomes dos atributos: {feature_names}\n")
print(f"Nomes das classes: {target_names}")

In [ ]:
## transforma NumPy Array para Pandas DataFrame
data_df = pd.DataFrame(X,columns=feature_names)

## sumariza os atributos numéricos (todos, neste caso)
data_df.describe()

###Fazendo a divisão dos dados com Holdout de 3 vias (treino/validação/teste)

In [ ]:
#Carregando funções específicas do scikit-learn

from sklearn.model_selection import train_test_split # função do scikit-learn que implementa um holdout
from sklearn.naive_bayes import GaussianNB # Naive Bayes Gaussiano
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report # métricas de desempenho
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # matriz de confusão

In [ ]:
## Exemplo de HOLDOUT de 2 vias: separa os dados em treino e teste, de forma estratificada (não utilizado aqui)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y) ## atenção: inicialmente, não mude o random_state para este exercício

In [ ]:
## HOLDOUT de 3 vias: separa os dados em treino e teste, de forma estratificada

## Definindo as proporções de treino, validação e teste.
train_ratio = 0.70
test_ratio = 0.15
validation_ratio = 0.15

## Fazendo a primeira divisão, para separar um conjunto de teste dos demais.
## Assuma X_temp e y_temp para os dados de treinamento+validação e X_test e y_test para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_ratio,random_state=42,stratify=y)

## Fazendo a segunda divisão, para gerar o conjunto de treino e validação a partir
## do conjunto de 'treinamento' da divisão anterior
## Assuma X_train e y_train para os dados de treinamento e X_valid e y_valid para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=validation_ratio/(train_ratio+test_ratio),random_state=42,stratify=y_temp)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)


**>> Analise e Discuta:**



1. Considere um cenário com 5 instâncias no conjunto de treinamento e  95 instâncias no conjunto de teste. O quão boa você acha que é a capacidade de generalização do modelo que provavelmente treinaremos?
    *   R: Baixa, mesmo que as 5 amostras sejam bem representativas da população, pois o modelo estará se baseando em em pouquíssimos dados para generalizar para muitos dados.

2. Sua resposta para 1 muda se tivermos 500 instâncias de treinamento e 9500 instâncias de teste?
    *   R: Sim, pois com 500 amostras bem representativas, o modelo tende a errar menos na generalização.

3.  Considere um cenário com 95 instâncias no conjunto de treinamento e 5 instâncias no conjunto de teste. O valor de desempenho do teste ainda é uma boa estimativa do poder de generalização?
    *   R: Talvez não seja o cenário ideal, porém com 95 instâncias de treino e 5 de teste, o modelo já consegue uma boa estimativa da sua capacidade de generalização, podendo porém, ficar com deficiência de exemplos extremos.

4. Sua resposta para 3 muda se tivermos 9500 instâncias de treinamento e 500 instâncias de teste?
    *   R: Sim, com 9500 amostras para treino e 500 para teste, é bem possível que nosso conjunto de dados englobe os mais diversos cenários, aumentando a capacidade de generalização do nosso modelo.

---

###Pré-processamento: Normalizando os dados

A normalização é feita de forma a evitar **Data Leakage** (vazamento de informações dos dados de teste durante o treinamento dos modelos). Os parâmetros para normalização são estimados a partir dos dados de treino, e posteriormente aplicados para normalizar todos os dados, isto é, treino, validação e teste.

A normalização é imprescindível para algoritmos baseados em distâncias, como o kNN.

In [ ]:
from sklearn.preprocessing import MinMaxScaler # função do scikit-learn que implementa normalização min-max

## O MinMaxScaler transformará os dados para que fiquem no intervalo [0,1] - importante para o kNN
scaler = MinMaxScaler()

## Iniciar a normalização dos dados. Primeiro fazer um 'fit' do scaler nos
## dados de treino. Esta etapa visa "aprender" os parâmetros para normalização.
## No caso do MinMaxScales, são os valores mínimos e máximos de cada atributo
scaler.fit(X_train)

## Aplicar a normalização nos três conjuntos de dados:
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

###Treinando um modelo Naïve Bayes Gaussiano (para dados numéricos)

In [ ]:
# Treinar Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Classificar dados no conjunto de teste
y_pred_nb = nb_model.predict(X_test)

# Avaliar o desempenho
print("Naive Bayes - Desempenho no Conjunto de Teste")
print(f"Acurácia: {accuracy_score(y_test, y_pred_nb):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_nb):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_nb):.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_nb))

In [ ]:
cm = confusion_matrix(y_test, y_pred_nb,labels=nb_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nb_model.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

###Treinando um modelo kNN - com otimização do hiperparâmetro k

In [ ]:
# Testar KNN com diferentes valores de k
# Conjunto de validação é usado para selecionar o melhor k
# Conjunto de teste é usado para avaliação final do modelo otimizado

# A análise é feita com a distância Euclidiana (padrão)
best_k = 1
best_score = 0

for k in range(1, 17,2):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train, y_train)

    # Classificar dados no conjunto de teste
    y_pred_valid = knn_model.predict(X_valid)

    # Acurácia no conjunto de validação
    score = accuracy_score(y_valid, y_pred_valid)
    print(f"K={k}: Acurácia na Validação = {score:.2f}")

    if score > best_score:
        best_score = score
        best_k = k

print(f"\nMelhor valor de K: {best_k} com Acurácia de {best_score:.2f} na Validação")

In [ ]:
# Avaliação final do KNN com o melhor k
knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(X_train, y_train) ## o ideal seria unir treino+validação neste treinamento, mas para fins de comparação entre modelos knn/NB mantive apenas X_train

# Classificar dados no conjunto de teste
y_pred_knn = knn_model.predict(X_test)

print(f"\nKNN - Desempenho com K={best_k} no Conjunto de Teste")
print(f"Acurácia: {accuracy_score(y_test, y_pred_knn):.2f}")
print(f"Precision: {precision_score(y_test, y_pred_knn):.2f}")
print(f"Recall: {recall_score(y_test, y_pred_knn):.2f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_knn))

In [ ]:
cm = confusion_matrix(y_test, y_pred_knn,labels=knn_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=knn_model.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

**>> Analise e Discuta:**

Com base nos resultados, qual dos dois modelos apresentou melhor desempenho geral no conjunto de teste, Naïve Bayes ou kNN?
Em quais classes o kNN obteve melhor sensibilidade (recall) e precisão do que o Naive Bayes, ou vice versa?

---

*   R: O modelo KNN com k=5 (acurácia = 0.97) apresentou desempenho superior ao modelo utilizando Naive Bayes Gaussiano (acurácia = 0.94). O valor de recall foi melhor no modelo de KNN com k=5 (recall = 0.98), enquanto que o modelo Naive Bayes ficou com recall = 0.94

###Analisando o impacto da divisão aleatória de dados no desempenho dos modelos




In [ ]:
# Inicializar listas para armazenar os resultados
accuracies = []
random_states = []

# Avaliar modelos (naïve Bayes/kNN) 30 vezes, variando o random_state
for i in range(30):
    random_state = np.random.randint(0, 1000)  # Gerar um random_state aleatório
    random_states.append(random_state)

    # Dividir os dados entre treino e teste (proporção fixa)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Normalizar dados
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # # Treinar o Naive Bayes
    # nb_model = GaussianNB()
    # nb_model.fit(X_train, y_train)

    # # Classificação e avaliação no conjunto de teste
    # y_pred_nb = nb_model.predict(X_test)
    # accuracy = accuracy_score(y_test, y_pred_nb)
    # accuracies.append(accuracy)

    # Treinar o kNN
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)

    # Classificação e avaliação no conjunto de teste
    y_pred_knn = knn_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_knn)
    accuracies.append(accuracy)


    # Exibir o desempenho a cada iteração
    print(f"Iteração {i+1}: Random State={random_state}, Acurácia={accuracy:.2f}")

# Plotar a variação das acurácias
plt.figure(figsize=(10,6))
plt.plot(range(1, 31), accuracies, marker='o', linestyle='--', color='b')
plt.title('Variação da Acurácia do Modelo em 30 Iterações com Random State Diferente')
plt.xlabel('Iteração')
plt.ylabel('Acurácia')
plt.xticks(range(1, 31))
plt.grid(True)
plt.show()

In [ ]:
# Amplitude dos resultados
max(accuracies) - min(accuracies)

**>> Analise e Discuta:**

Observe a variação do valor de *random_state* na divisão dos dados e os respectivos resultados do desempenho na classificação. Como isso afeta os resultados?  Explique o impacto de diferentes divisões dos dados de treino/validação/teste no desempenho dos modelos. Por que é importante repetir os experimentos várias vezes, variando o random_state? O que a repetição traz em termos de confiabilidade dos resultados?

---

*   R: O valor de `random_state` determina a "seed" que será usada para embaralhar os dados, e este valor é um número aleatório, o que introduz um potencial viés no modelo, visto que a aleatoriedade sem repetição tem como risco inerente a possibilidade de dificultar a boa representatividade da população. Uma boa forma de termos alta representatividade da população é mantermos a aleatoriedade, porém repetidas vezes, fazendo com que a lei dos grandes números entre em ação e com isso nós podemos esperar que a média das amostras se aproxime cada vez mais da média real.

###Analisando o impacto do tamanho do conjunto de teste na avaliação de desempenho dos modelos



In [ ]:
# Inicializar listas para armazenar resultados
variances = []
amplitudes = []
avg_accuracies = []

# Definir as proporções de conjunto de teste
test_sizes = np.arange(0.05, 0.70, 0.05)

# Loop para cada proporção de conjunto de teste
for test_size in test_sizes:
    accuracies = []

    # Repetir o experimento 30 vezes para cada tamanho de conjunto de teste
    for i in range(30):
        random_state = np.random.randint(0, 1000)

        # Dividir os dados com a proporção especificada para o conjunto de teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        # Normalizar dados
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # # Treinar o Naive Bayes
        # nb_model = GaussianNB()
        # nb_model.fit(X_train, y_train)

        # # Previsão e avaliação no conjunto de teste
        # y_pred_nb = nb_model.predict(X_test)
        # accuracy = accuracy_score(y_test, y_pred_nb)
        # accuracies.append(accuracy)

        # Treinar o kNN
        knn_model = KNeighborsClassifier(n_neighbors=5)
        knn_model.fit(X_train, y_train)

        # Previsão e avaliação no conjunto de teste
        y_pred_knn = knn_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred_knn)
        accuracies.append(accuracy)

    # Calcular variância, amplitude e média dos desempenhos
    variance = np.var(accuracies)
    amplitude = np.max(accuracies) - np.min(accuracies)
    avg_accuracy = np.mean(accuracies)

    # Armazenar os resultados
    variances.append(variance)
    amplitudes.append(amplitude)
    avg_accuracies.append(avg_accuracy)

    # Exibir os resultados intermediários
    print(f"Tamanho do Conjunto de Teste: {test_size*100:.1f}%")
    print(f"   Média da Acurácia: {avg_accuracy:.3f}")
    print(f"   Variância: {variance:.5f}")
    print(f"   Amplitude (Máx - Mín): {amplitude:.3f}")
    print("")

# Gráfico: Variação da Acurácia com Diferentes Tamanhos de Conjunto de Teste
plt.figure(figsize=(10,6))
plt.plot(test_sizes * 100, avg_accuracies, marker='o', linestyle='--', color='b', label="Média das Acurácias")
plt.title('Média da Acurácia do KNN com Diferentes Tamanhos de Conjunto de Teste')
plt.xlabel('Tamanho do Conjunto de Teste (%)')
plt.ylabel('Média da Acurácia')
plt.grid(True)
plt.legend()
plt.show()

# # Gráfico: Variância vs Tamanho do Conjunto de Teste
# plt.figure(figsize=(10,6))
# plt.scatter(test_sizes * 100, variances, color='r', label="Variância do Desempenho")
# plt.title('Variância do Desempenho vs Tamanho do Conjunto de Teste')
# plt.xlabel('Tamanho do Conjunto de Teste (%)')
# plt.ylabel('Variância do Desempenho')
# plt.grid(True)
# plt.legend()
# plt.show()

# Gráfico: Amplitude vs Tamanho do Conjunto de Teste
plt.figure(figsize=(10,6))
plt.scatter(test_sizes * 100, amplitudes, color='g', label="Amplitude (Máx - Mín)")
plt.title('Amplitude do Desempenho vs Tamanho do Conjunto de Teste')
plt.xlabel('Tamanho do Conjunto de Teste (%)')
plt.ylabel('Amplitude do Desempenho')
plt.grid(True)
plt.legend()
plt.show()

**>> Analise e Discuta:**

Conforme o tamanho do conjunto de teste aumenta, como muda a variância no desempenho do modelo? Por que esse comportamento ocorre? Ao comparar a amplitude (diferença entre o máximo e o mínimo) do desempenho em diferentes tamanhos de conjunto de teste, o que você observa? Qual é a relação entre o tamanho do teste e a amplitude dos resultados?

Se a variância dos resultados de acurácia for muito alta, o que isso pode indicar sobre o seu modelo ou sobre a forma como os dados estão sendo divididos?

---

*   R: Conforme o tamanho do conjunto de teste aumenta, a variância no desempenho do modelo tende a diminuir. Isso ocorre pois com mais amostras no conjunto de teste, a influência de cada observação se torna menor, aumentando a estabilidade do modelo. Quanto a amplitude do modelo, conforme aumentamos o tamanho do conjunto de teste, a amplitude tende a diminuir, aumentando a estabilidade. No caso de variâncias muito altas nos resultados, isso pode indicar um possível overfitting, bem como a representatividade precárias nos dados de treinamento. Também é possível verificar essa alta variância de acurácia em modelos muito sensíveis aos dados de entrada.

###Analisando o impacto do tamanho do conjunto de treino na avaliação de desempenho dos modelos



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Função para rodar o KNN com diferentes tamanhos de treino
def run_knn_analysis(X, y, test_size=0.1, train_sizes=[0.1, 0.2, 0.3, 0.4, 0.50, 0.6, 0.7, 0.8], iterations=10):
    results = {}

    # Fixando o conjunto de teste
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Criando o objeto de normalização
    scaler = StandardScaler()
    scaler.fit(X_train_full)

    # Normalizando o conjunto de teste
    X_test = scaler.transform(X_test)

    for train_size in train_sizes:
        accuracies = []
        for _ in range(iterations):
            # Amostrando um conjunto de treino de tamanho variável
            X_train_sample, _, y_train_sample, _ = train_test_split(
                X_train_full, y_train_full, train_size=train_size, random_state=None
            )

            # Normalizando o conjunto de treino amostrado
            X_train_sample = scaler.transform(X_train_sample)

            # Treinar o kNN
            knn_model = KNeighborsClassifier(n_neighbors=5)
            knn_model.fit(X_train_sample, y_train_sample)

            # Classificação e avaliação no conjunto de teste fixo
            y_pred_knn = knn_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred_knn)
            accuracies.append(accuracy)

        # Calculando estatísticas do desempenho
        variance = np.var(accuracies)
        amplitude = np.max(accuracies) - np.min(accuracies)
        average = np.mean(accuracies)

        results[train_size] = {
            'accuracies': accuracies,
            'variance': variance,
            'amplitude': amplitude,
            'average': average
        }

    return results

# Exemplo de uso
variances = []
amplitudes = []
averages = []
train_sizes = [0.1, 0.2, 0.3, 0.4, 0.50, 0.6, 0.7, 0.8]

results = run_knn_analysis(X, y)

for train_size, metrics in results.items():
    variances.append(metrics['variance'])
    amplitudes.append(metrics['amplitude'])
    averages.append(metrics['average'])

# Gráfico de variância vs tamanho do conjunto de treino
plt.figure(figsize=(10, 6))
plt.scatter(train_sizes, variances, c='blue', label='Variância do Desempenho')
plt.plot(train_sizes, variances, color='blue', linestyle='--')
plt.xlabel('Proporção do Conjunto de Treino')
plt.ylabel('Variância da Acurácia')
plt.title('Variância da Acurácia em Função do Tamanho do Conjunto de Treino')
plt.legend()
plt.grid(True)
plt.show()

# Gráfico de amplitude vs tamanho do conjunto de treino
plt.figure(figsize=(10, 6))
plt.scatter(train_sizes, amplitudes, c='red', label='Amplitude do Desempenho (Max - Min)')
plt.plot(train_sizes, amplitudes, color='red', linestyle='--')
plt.xlabel('Proporção do Conjunto de Treino')
plt.ylabel('Amplitude da Acurácia')
plt.title('Amplitude da Acurácia em Função do Tamanho do Conjunto de Treino')
plt.legend()
plt.grid(True)
plt.show()

# Gráfico de média vs tamanho do conjunto de treino
plt.figure(figsize=(10, 6))
plt.scatter(train_sizes, averages, c='b', label='Média')
plt.plot(train_sizes, averages, color='b', linestyle='--')
plt.xlabel('Proporção do Conjunto de Treino')
plt.ylabel('Média da Acurácia')
plt.title('Média da Acurácia em Função do Tamanho do Conjunto de Treino')
plt.legend()
plt.grid(True)
plt.show()

**>> Analise e Discuta:**

Mantendo o conjunto de teste fixo, que mudança ou tendência observamos no desempenho dos modelos conforme mais dados de treino são utilizados? Como a amplitude (diferença entre o máximo e o mínimo) do desempenho varia com o tamanho do conjunto de treino? O que isso nos diz sobre a confiabilidade do modelo com tamanhos pequenos de conjunto de treino?

---

*   R: Embora no conjunto de dados observado a menor proporção de dados no conjunto de treino resultou em uma média de acurácia superior, isto não é o padrão que observamos ao analisar a maioria dos conjuntos de dados. Quando temos menos dados para treinar um determinado modelo, estamos limitando a sua capacidade de representação, o que implica diretamente na sua acurácia. Conforme aumentamos o conjunto de treino, percebemos a diminuição na amplitude e variância da acurácia, o que indica que o modelo está indo em direção a estabilidade de desempenho, garantindo maior confiabilidade para aplicar este modelo em dados ainda não observados.

Sumarize as suas análises e conclusões em um documento word ou PDF (inserindo as respectivas perguntas propostas), e envie no Moodle.